In [1]:
import os, sys
from sklearn.pipeline import Pipeline
from pathlib import Path

# Add local codebase to path
home = str(Path.home())
sys.path.insert(1, f'{home}/Documents/woonfraude/codebase')

# Import own modules
from datasets_oo import *
from clean_oo import *
from extract_features_oo import *

/home/tjongstra/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# bagDataset = BagDataset()
# bagDataset.download(force=True)
# bagDataset.load('download')
# bagDataset.bag_fix()

In [2]:
# adresDataset = AdresDataset()
# adresDataset.load('download_leegstand_woningId_bag_personen_hotline')

In [2]:
###################
## Load datasets ##
###################
# Download (or load cached versions of) the datasets.

# Initialize dataset objects
adresDataset = AdresDataset()
zakenDataset = ZakenDataset()
stadiaDataset = StadiaDataset()
personenDataset = PersonenDataset()
bagDataset = BagDataset()
hotlineDataset = HotlineDataset()

# # Download and perform dataset-specific processing.
# # Uncomment code and run once. Use load-from cache functionality below afterwards.
# adresDataset.download(force=True)
# adresDataset.load('download')
# adresDataset.extract_leegstand()
# adresDataset.enrich_with_woning_id()

# zakenDataset.download(force=True)
# zakenDataset.load('download')
# zakenDataset.add_categories()
# zakenDataset.filter_categories()

# stadiaDataset.download(force=True)
# stadiaDataset.load('download')
# stadiaDataset.add_zaak_stadium_ids()
# stadiaDataset.add_labels()

# personenDataset.download(force=True)

# bagDataset.download(force=True)
bagDataset.load('download')
bagDataset.bag_fix()

# hotlineDataset.download(force=True)
# hotlineDataset.load('download')

# Load datasets from cache (if download and processing steps have already been done).
adresDataset.load('download_leegstand_woningId')
zakenDataset.load('download_categories_filterCategories')
stadiaDataset.load('download_ids_labels')
personenDataset.load('download')
bagDataset.load('download_columnFix')
hotlineDataset.load('download')

Version 'download' of dataset 'bag' loaded!
Saving version 'download_columnFix' of dataframe 'bag'.


/home/tjongstra/.local/lib/python3.6/site-packages/pandas/core/generic.py:2378: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['id_nummeraanduiding', 'landelijk_id_nummeraanduiding', 'huisnummer_toevoeging_nummeraanduiding', 'postcode', 'type', 'adres_nummer', 'vervallen_nummeraanduiding', 'hoofdadres', '_openbare_ruimte_naam_nummeraanduiding', 'bron_id_nummeraanduiding', 'ligplaats_id', 'openbare_ruimte_id', 'standplaats_id', 'status_id_nummeraanduiding', 'verblijfsobject_id', '_geom', 'id_ligplaats', 'landelijk_id_ligplaats', 'vervallen_ligplaats', 'geometrie_ligplaats', '_huisletter_ligplaats', 'bron_id_ligplaats', 'status_id_ligplaats', 'id_standplaats', 'landelijk_id_standplaats', 'vervallen_standplaats', 'geometrie_standplaats', '_huisnummer_toevoeging_standplaats', 'bron_id_standplaats', 'status_id_standplaats', 'id_verblijfsobject', 'landelijk_id_verb

Version 'download_leegstand_woningId' of dataset 'adres' loaded!
Version 'download_categories_filterCategories' of dataset 'zaken' loaded!
Version 'download_ids_labels' of dataset 'stadia' loaded!
Version 'download' of dataset 'personen' loaded!
Version 'download_columnFix' of dataset 'bag' loaded!
Version 'download' of dataset 'hotline' loaded!


In [4]:
#########################
## Clean zaken dataset ##
#########################

zakenPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=zakenDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['begindatum','einddatum', 'wzs_update_datumtijd'],
        clean_dates=True,
        lower_string_columns=True,
        impute_missing_values=True)
    ),
    ('extract', FeatureExtractionTransformer(
        categorical_cols_hot=['afg_code_beh', 'beh_code', 'eigenaar', 'categorie'])
    )
    ])

zaken = zakenPipeline.fit_transform(zakenDataset.data)

Dataframe "zaken": Dropped 0 duplicates!
Dataframe "zaken": Fixed dates!
Dataframe "zaken": Cleaned out 3 dates!
Lowered strings of cols ['beh_code', 'beh_oms', 'afg_code_beh', 'afs_code', 'afs_oms', 'afg_code_afs', 'eigenaar', 'zaak_id', 'mededelingen', 'categorie'] in df zaken!
Now extracting features from column: 'afg_code_beh'.
Done!
Now extracting features from column: 'beh_code'.
Done!
Now extracting features from column: 'eigenaar'.
Done!
Now extracting features from column: 'categorie'.
Done!


In [5]:
##########################
## Clean stadia dataset ##
##########################

stadiaPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=stadiaDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['begindatum', 'peildatum', 'einddatum', 'date_created',
                          'date_modified', 'wzs_update_datumtijd'],
        clean_dates=True,
        lower_string_columns=True,
        impute_missing_values=True)
    )])

stadia = stadiaPipeline.fit_transform(stadiaDataset.data)

Dataframe "stadia": Dropped 0 duplicates!
Dataframe "stadia": Fixed dates!
Dataframe "stadia": Cleaned out 0 dates!
Lowered strings of cols ['afg_co', 'sta_code', 'sta_oms', 'afg_code_stad', 'afs_code', 'afs_oms', 'afg_code_afs', 'resultaat', 'mdr_code', 'user_created', 'user_modified', 'stadia_id', 'zaak_id', 'stadium_id', 'label'] in df stadia!


In [6]:
############################
## Clean personen dataset ##
############################

personenPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=personenDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['geboortedatum'],
        lower_string_columns=True)
    )])

personen = personenPipeline.fit_transform(personenDataset.data)

Dataframe "personen": Dropped 0 duplicates!
Dataframe "personen": Fixed dates!
Lowered strings of cols ['pen_type', 'gezinsverhouding', 'geslacht', 'voorletters', 'burgerlijke_staat', 'naam', 'geheim_adres', 'voorv_mnaam', 'voorv_naam', 'meisjesnaam', 'vertrekdatum_adam', 'ind_naamgebruik', 'nat_ned', 'ind_nat_ovlp', 'verblijfstatus', 'datum_einde_vblstat', 'landcode', 'user_created', 'user_modified', 'datum_begin_vblstat', 'ais_nr', 'crv_nr', 'geheim', 'in_onderzoek', 'datum_verkrijging_vreemd', 'voorletters_zdia', 'naam_zdia', 'voorv_mnaam_zdia', 'voorv_naam_zdia', 'meisjesnaam_zdia', 'nm_dia_255', 'mnm_dia_255'] in df personen!


In [7]:
list(bagDataset.data.columns)

['id_nummeraanduiding',
 'landelijk_id_nummeraanduiding',
 'huisnummer_toevoeging_nummeraanduiding',
 'postcode',
 'type',
 'adres_nummer',
 'vervallen_nummeraanduiding',
 'date_modified_nummeraanduiding',
 'hoofdadres',
 '_openbare_ruimte_naam_nummeraanduiding',
 'bron_id_nummeraanduiding',
 'ligplaats_id',
 'openbare_ruimte_id',
 'standplaats_id',
 'status_id_nummeraanduiding',
 'verblijfsobject_id',
 '_geom',
 'id_ligplaats',
 'landelijk_id_ligplaats',
 'vervallen_ligplaats',
 'geometrie_ligplaats',
 '_huisnummer_ligplaats',
 '_huisletter_ligplaats',
 'bron_id_ligplaats',
 'status_id_ligplaats',
 'id_standplaats',
 'landelijk_id_standplaats',
 'vervallen_standplaats',
 'geometrie_standplaats',
 '_huisnummer_toevoeging_standplaats',
 'bron_id_standplaats',
 'status_id_standplaats',
 'id_verblijfsobject',
 'landelijk_id_verblijfsobject',
 'oppervlakte',
 'bouwlaag_toegang',
 'status_coordinaat_code',
 'status_coordinaat_omschrijving',
 'verhuurbare_eenheden',
 'bouwlagen',
 'type_woon

In [ ]:
#######################
## Clean BAG dataset ##
#######################

bagPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=bagDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=[],
        drop_columns = ['indicatie_geconstateerd', 'indicatie_in_onderzoek', 'woningvoorraad'],
        lower_string_columns=True,
        impute_missing_values=True,
        impute_missing_values_mode=['status_coordinaat_code'],
#         impute_missing_values_mode=['status_coordinaat_code', 'indicatie_geconstateerd',
#                                     'indicatie_in_onderzoek', 'woningvoorraad'],
        fillna_columns={'_huisnummer_ligplaats': 0,
                         '_huisletter_ligplaats': 'None',
                         '_openbare_ruimte_naam_verblijfsobject': 'None',
                         '_huisnummer_toevoeging_standplaats': 'None',
                         'type_woonobject_omschrijving': 'None',
                         'eigendomsverhouding_id': 'None',
                         'financieringswijze_id': -1,
                         'gebruik_id_nummeraanduiding': -1,
                         'reden_opvoer_id': -1,
                         'status_id_verblijfsobject': -1,
                         'toegang_id': 'None'})
    ),
    ('extract', FeatureExtractionTransformer(
        categorical_cols_hot=['status_coordinaat_code', 'type_woonobject_omschrijving',
                              'eigendomsverhouding_id', 'financieringswijze_id',
                              'gebruik_id', 'ligging_id', 'reden_opvoer_id',
                              'status_id_nummeraanduiding', 'toegang_id'])
    )
    ])

bag = bagPipeline.fit_transform(bagDataset.data)

Dataframe "bag": Dropped 0 duplicates!


In [9]:
list(bag.columns)

['document_mutatie@bag',
 'document_nummer@bag',
 'begin_geldigheid@bag',
 'einde_geldigheid@bag',
 'id_nummeraanduiding@bag',
 'landelijk_id_nummeraanduiding@bag',
 'huisnummer_toevoeging@bag',
 'postcode@bag',
 'type@bag',
 'adres_nummer@bag',
 'vervallen_nummeraanduiding@bag',
 'date_modified_nummeraanduiding@bag',
 'hoofdadres@bag',
 '_openbare_ruimte_naam_nummeraanduiding@bag',
 'bron_id_nummeraanduiding@bag',
 'ligplaats_id@bag',
 'openbare_ruimte_id@bag',
 'standplaats_id@bag',
 'verblijfsobject_id@bag',
 '_geom@bag',
 'id_ligplaats@bag',
 'landelijk_id_ligplaats@bag',
 'vervallen_ligplaats@bag',
 'geometrie_nummeraanduiding@bag',
 '_huisnummer@bag',
 '_huisletter@bag',
 '_huisnummer_toevoeging_nummeraanduiding@bag',
 '_gebiedsgerichtwerken_id@bag',
 '_grootstedelijkgebied_id@bag',
 'bron_id_ligplaats@bag',
 'buurt_id@bag',
 'status_id_ligplaats@bag',
 'id_standplaats@bag',
 'landelijk_id_standplaats@bag',
 'vervallen_standplaats@bag',
 'geometrie_ligplaats@bag',
 'bron_id_stand

In [10]:
###########################
## Clean hotline dataset ##
###########################

hotlinePipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=hotlineDataset.id_column,
        drop_duplicates=True,
        lower_string_columns=True,
        impute_missing_values=True)
    )])

hotline = hotlinePipeline.fit_transform(hotlineDataset.data)

Dataframe "hotline": Dropped 0 duplicates!
Lowered strings of cols ['mdw_code', 'overtreding_code', 'melder_anoniem', 'melder_naam', 'melder_emailadres', 'melder_telnr', 'situatie_schets', 'user_created', 'user_modified'] in df hotline!


In [11]:
#########################
## Clean adres dataset ##
#########################

# Hier de extract stap weghalen? Deze past waarschijnlijk beter na het combinen v/d datasets.
adresPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=adresDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['hvv_dag_tek', 'max_vestig_dtm', 'wzs_update_datumtijd'],
        lower_string_columns=True,
        impute_missing_values=True,
        fillna_columns={'hsnr': 0, 'sttnaam': 'None', 'hsltr': 'None', 'toev': 'None'})
    ),
    ('extract', FeatureExtractionTransformer(
        categorical_cols_hot=['toev', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr'],
        ))
    ])

adres = adresPipeline.fit_transform(adresDataset.data)

Dataframe "adres": Dropped 0 duplicates!
Dataframe "adres": Fixed dates!
Lowered strings of cols ['postcode', 'sdl_code', 'brt_code', 'pvh_cd', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr', 'wzs_buurtcode_os_2015', 'wzs_buurtnaam_os_2015', 'wzs_buurtcombinatiecode_os_2015', 'wzs_buurtcombinatienaam_os_2015', 'wzs_22gebiedencode_os_2015', 'wzs_22gebiedennaam_os_2015', 'wzs_rayoncode_os_2015', 'wzs_rayonnaam_os_2015', 'wzs_stadsdeelcode_os_2015', 'wzs_stadsdeelnaam_os_2015', 'wzs_alternatieve_buurtennaam_os_2015', 'wzs_alternatieve_buurtencode_os_2015', 'wzs_geom', 'wzs_wijze_verrijking_geo', 'wzs_22gebiedencode_2015', 'wzs_22gebiedennaam_2015', 'sttnaam', 'hsltr', 'toev', 'brtcombi_naam', 'sdl_naam', 'brt_naam', 'a_dam_bag', 'landelijk_bag', 'hvv_dag_tek', 'max_vestig_dtm'] in df adres!
Now extracting features from column: 'toev'.
Done!
Now extracting features from column: 'pvh_omschr'.
Done!
Now extracting features from column: 'sbw_omschr'.
Done!
Now extracting features from column: 'sbv_

In [12]:
######################
## Combine datasets ##
######################

adresDataset.enrich_with_bag(bagDataset.data)
adresDataset.enrich_with_personen_features(personenDataset.data)
adresDataset.add_hotline_features(hotlineDataset.data)

adres_remove = [# Remove because cols do not exists when melding is received
                    'wzs_update_datumtijd',
                    # Remove because cols do not add extra information.
                    'kmrs',
                    'straatcode',
                    'xref',
                    'yref',
                    'postcode',
                    'wzs_buurtcode_os_2015',
                    'wzs_buurtcombinatiecode_os_2015',
                    'wzs_stadsdeelcode_os_2015',
                    'sttnaam',
                    'hvv_dag_tek', # Empty column
                    'max_vestig_dtm', # Empty column
                    'wzs_22gebiedencode_os_2015', # Empty column
                    'wzs_22gebiedennaam_os_2015', # Empty column
                    'sdl_naam',
                    'pvh_cd',
                    'sbv_code',
                    'sbw_code',
                    'wzs_wijze_verrijking_geo',
                    'wzs_22gebiedencode_2015',
                    'brt_naam',
                    'wzs_buurtnaam_os_2015',
                    'wzs_buurtcombinatienaam_os_2015',
                    'wzs_rayonnaam_os_2015',
                    'wzs_rayoncode_os_2015',
                    'wzs_stadsdeelnaam_os_2015',
                    'wzs_alternatieve_buurtennaam_os_2015',
                    'wzs_alternatieve_buurtencode_os_2015',
                    'hsltr',
                    'wzs_geom',
                    'brt_code',
                    'brtcombi_code',
                    'brtcombi_naam',
                    'sdl_code',
                    'wzs_22gebiedennaam_2015',
                    'wzs_id',
                    'a_dam_bag',
                    'landelijk_bag']

bag_remove = ['einde_geldigheid@bag',               # Only 2 entries in column.
              'verhuurbare_eenheden@bag',           # Only ~2k entries in column.
              'geometrie_ligplaats@bag',            # Needs a lot of processing before being useful.
              'bron_id@bag',                        # Only 2 entries in column.
              'locatie_ingang_id@bag',              # Only 2 entries in column.
              'reden_afvoer_id@bag',                # Only a few entries in column.
              '_gebiedsgerichtwerken_id@bag',       # Superfluous (gebied).
              '_grootstedelijkgebied_id@bag',       # Superfluous (grootstedelijkgebied).
              'buurt_id@bag',                       # Superfluous (buurt).
              # ONDERSTAANDE 4 KOLOMMEN KONDEN EERDER NIET WEG IVM MATCH MET ADRES DATAFRAME.
              # DEZE MOETEN NU WEL WEG, DAAROM WORDT NU HIER ALLES WEGGEHAALD.
              '_openbare_ruimte_naam_verblijfsobject@bag',          # Superfluous (straatnaam).
              '_huisnummer@bag',                    # Superfluous (huisnummer).
              '_huisletter@bag',                    # Superfluous (huisletter).
              '_huisnummer_toevoeging@bag',         # Superfluous (huisnummer toevoeging).
              'vervallen@bag',                      # Superfluous (all values in col are equal).
              'document_mutatie@bag',               # Not available at time of signal.
              'date_modified@bag',                  # Not available at time of signal.
              'document_nummer@bag',                # Not needed? (Swaan?)
              'status_coordinaat_omschrijving@bag', # Not needed? (Swaan?)
              'type_woonobject_code@bag',           # Not needed? (Swaan?)
              'id_ligplaats@bag',                   # Not needed.
              'landelijk_id_ligplaats@bag',         # Not needed.
              'id_standplaats@bag',                 # Not needed.
              'landelijk_id_standplaats@bag',       # Not needed.
              'id_verblijfsobject@bag',             # Not needed.
              'landelijk_id_verblijfsobject@bag',   # Not needed.
              ]

# Remove adres_id, since this is not a feature we want our algorihtm to try and learn from.
adresDataset.data.drop(columns=adres_remove + bag_remove + ['adres_id'], inplace=True)

KeyError: 'huisnummer_toevoeging_nummeraanduiding@bag'

In [11]:
list(adresDataset.data.columns)

['adres_id',
 'straatcode',
 'postcode',
 'sdl_code',
 'brt_code',
 'pvh_cd',
 'pvh_omschr',
 'sbw_omschr',
 'sbv_omschr',
 'wzs_id',
 'wzs_buurtcode_os_2015',
 'wzs_buurtnaam_os_2015',
 'wzs_buurtcombinatiecode_os_2015',
 'wzs_buurtcombinatienaam_os_2015',
 'wzs_22gebiedencode_os_2015',
 'wzs_22gebiedennaam_os_2015',
 'wzs_rayoncode_os_2015',
 'wzs_rayonnaam_os_2015',
 'wzs_stadsdeelcode_os_2015',
 'wzs_stadsdeelnaam_os_2015',
 'wzs_alternatieve_buurtennaam_os_2015',
 'wzs_alternatieve_buurtencode_os_2015',
 'wzs_update_datumtijd',
 'wzs_geom',
 'wzs_lon',
 'wzs_lat',
 'wzs_wijze_verrijking_geo',
 'wzs_22gebiedencode_2015',
 'wzs_22gebiedennaam_2015',
 'sttnaam',
 'hsnr',
 'hsltr',
 'toev',
 'brtcombi_code',
 'brtcombi_naam',
 'sdl_naam',
 'xref',
 'yref',
 'brt_naam',
 'a_dam_bag',
 'landelijk_bag',
 'sbw_code',
 'sbv_code',
 'inwnrs',
 'kmrs',
 'hvv_dag_tek',
 'max_vestig_dtm',
 'leegstand',
 'ads_id',
 'wng_id',
 'document_mutatie@bag',
 'document_nummer@bag',
 'begin_geldigheid@ba